----------------------------------------------------
# **Introduction to Optimization - Project.**
----------------------------------------------------

In [ ]:
# Import the standard libraries of pandas.

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings. filterwarnings("ignore")
sns.set_style('whitegrid')
from google.colab import files

In [ ]:
# Install the solver and import its libraries, in addition import all the
# libraries with which we will prepare the features.

!pip3 install ortools 
!pip install python-igraph

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import numpy as np
import time
import random 
from random import randrange
from scipy import stats
from scipy.stats import skew
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse.csgraph import depth_first_tree
from igraph import Graph, mean
import igraph
import itertools
import math

     |████████████████████████████████| 14.0MB 322kB/s 
     |████████████████████████████████| 133kB 59.9MB/s 
     |████████████████████████████████| 1.0MB 51.5MB/s 
ERROR: tensorflow-metadata 0.26.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


     |████████████████████████████████| 3.2MB 7.6MB/s 


## **In this section we create an instance of TSP**
## **solve it and produce for it the following features:**
* **Mean** - Average weights of the distance matrix.
* **Std** - Standard Deviation of the distance matrix.
* **Skewness** - What is the tendency of the weights in the distance matrix.
* **Noc** - Number of cities we have in the distance matrix [matrix dimension].
* **Td** - The total distance of the solution rout.
* **Dmft** - Distance matrix features time, That is how long it took us to calculate all these features.
* **MST_Mean** - Average weights of the MST.
* **MST_Std** - Standard Deviation of the MST.
* **MST_Skewness** - What is the tendency of the weights in the MST.
* **MST_ft** - MST features time, That is how long it took us to calculate the MST & all these features.
* **D_Mean** - Average degree of the MST.
* **D_Std** - Standard Deviation of the MST degrees.
* **D_Skewness** - What is the tendency of the degrees in the MST.
* **DFT_Mean** - The average weight of the deepest track in MST.
* **DFT_Std** - Standard Deviation of the deepest track in MST.
* **DFT_Max** - The heaviest arch on the longest route in MST.
* **DDFT_ft** - Degree & DFT features time, That is how long it took us to calculate all these features.



In [ ]:
# Simple travelling salesman problem between cities - solver OR Tools By Google. 

def create_data_model():
    # Stores the data for the problem. 
    data = {}
    # dim will be the number of Vertices\Cities in the Traveling Salesman Problem.
    # Randomly select the matrix dimension in unifom distribution.
    dim = np.random.randint(10, 350) 
    # Generate a square symmetric matrix It will be the distance matrix that the solver will solve.
    square_matrice = [[0 for row in range(dim)] for col in range(dim)]
    for i in range(dim):
        for j in range(dim):
            if i == j:
                square_matrice[i][j] = 0
            else:
                # Randomly fill the matrix in unifom distribution.
                square_matrice[i][j] = square_matrice[j][i] = np.random.randint(1, 1000) 
    data['distance_matrix'] = square_matrice # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def main():
    # Start measuring solution time.
    start_time = time.time()
    # Instantiate the data problem.
    data = create_data_model()
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    def distance_callback(from_index, to_index):
        # Returns the distance between the two nodes. 
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    solution_time = time.time() - start_time

    '''In this part of the code we will create the following features on the distance matrix of the problem.
    *  Mean - Average weights of the distance matrix.
    *  Std - Standard Deviation of the distance matrix.
    *  Skewness  - What is the tendency of the weights in the distance matrix.
    *  Noc - Number of cities we have in the distance matrix [matrix dimension].
    *  Td - The total distance of the solution rout. 
    *  Dmft - Distance matrix features time, That is how long it took us to calculate all these features.
    '''
    dmt_start_time = time.time()
    mat = np.array(data['distance_matrix'])
    mean = mat.mean()
    std = mat.std()
    merged = list(itertools.chain(*mat))
    skewness = skew(merged)
    noc = len(data['distance_matrix'])
    td = solution.ObjectiveValue() if solution else -1
    dmft = time.time() - dmt_start_time

    '''In this part of the code we will create from the distance matrix of the problem an MST and than
    on the MST we take the following features.
    *  MST_Mean - Average weights of the MST.
    *  MST_Std - Standard Deviation of the MST.
    *  MST_Skewness  - What is the tendency of the weights in the MST.
    *  MST_ft - MST features time, That is how long it took us to calculate the MST & all these features.
    '''
    spt_start_time = time.time()
    X = csr_matrix(mat)
    Tcsr = minimum_spanning_tree(X)
    mat_st = np.array(Tcsr.toarray().astype(int))
    mst_mean = mat_st.mean()
    mst_std = mat_st.std()
    merged_st = list(itertools.chain(*mat_st))
    mst_skewness = skew(merged_st)
    mst_ft = time.time() - spt_start_time

    '''In this part of the code we calculate features from the MST that are considered to be
    related to the rank and depth of the tracks in it.
    *  D_Mean - Average degree of the MST.
    *  D_Std - Standard Deviation of the MST degrees.
    *  D_Skewness  - What is the tendency of the degrees in the MST.
    *  DFT_Mean - The average weight of the deepest track in MST.
    *  DFT_Std - Standard Deviation of the deepest track in MST.
    *  DFT_Max - The heaviest arch on the longest route in MST.
    *  DDFT_ft - Degree & DFT features time, That is how long it took us to calculate all these features.
    '''
    dstt_start_time = time.time()
    g = Graph.Weighted_Adjacency(mat_st.tolist())
    d_mean = igraph.statistics.mean(g.degree())
    d_std = igraph.statistics.sd(g.degree())
    d_skewness = skew(g.degree())
    d_t = depth_first_tree(X, 0, directed=False)
    mat_dt = np.array(d_t.toarray().astype(int))
    dft_mean = mat_dt.mean()
    dft_std = mat_dt.std()
    dft_max = np.amax(mat_dt)
    ddft_ft = time.time() - dstt_start_time

    # In this map we will hold all the features and their results.
    features_map = {'Mean': mean, 'Std': std, 'Skewness': skewness, 'Noc': noc, 'Td': td, 'Dmft': dmft,
                    'MST_Mean': mst_mean, 'MST_Std': mst_std, 'MST_Skewness': mst_skewness, 'MST_ft': mst_ft,
                    'D_Mean': d_mean, 'D_Std': d_std, 'D_Skewness': d_skewness, 'DFT_Mean': dft_mean,'DFT_Std': dft_std,
                    'DFT_Max': dft_max, 'DDFT_ft': ddft_ft, 'Solution_time': solution_time}

    return features_map

In [ ]:
# Main    
# Create dataFrame.
data_TSP = pd.DataFrame()
# Fill the dataFrame.
for i in range(10000):
    #print(i)
    features_map = main()
    data_TSP = data_TSP.append(features_map, ignore_index=True) 

In [ ]:
# Show data frame.

data_TSP.head()

,DDFT_ft,DFT_Max,DFT_Mean,DFT_Std,D_Mean,D_Skewness,D_Std,Dmft,MST_Mean,MST_Skewness,MST_Std,MST_ft,Mean,Noc,Skewness,Solution_time,Std,Td
0,0.002684,994.0,4.281241,53.119427,1.983051,0.991500,1.132149,0.004528,0.079216,17.508554,1.068120,0.005651,503.054007,118.0,-0.039861,0.269456,292.236114,2616.0
1,0.004046,997.0,2.391274,39.626311,1.990148,1.196133,1.067013,0.010491,0.031838,21.608326,0.541832,0.013047,500.059453,203.0,-0.014632,1.373102,289.733257,3185.0
2,0.006801,994.0,1.732715,34.118103,1.993127,0.854827,1.003419,0.020711,0.016698,31.544559,0.363748,0.029591,499.979240,291.0,-0.010912,2.116377,288.761128,3643.0
3,0.007278,995.0,1.592935,32.389269,1.993528,1.213399,1.122363,0.023446,0.013615,30.368070,0.297845,0.033800,498.676407,309.0,0.003925,2.001439,290.388494,3714.0
4,0.006666,999.0,1.688561,33.446180,1.993220,1.417943,1.121810,0.021590,0.014318,28.048382,0.302933,0.027265,496.423625,295.0,0.012855,2.972457,289.502941,3534.0


In [ ]:
data_TSP.to_csv('data_10000.csv')
files.download('data_10000.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>